<a href="https://colab.research.google.com/github/caprolaliac/Aimore/blob/main/sentiment-analysis/emotion_based_sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch numpy pandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.0 MB/s eta 0:00:00


In [13]:
import torch
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import pipeline

In [3]:
df=pd.read_csv('/content/drive/MyDrive/The Acolyte-reviews.csv')
df.head()

,Review Title,Review Date,Review Content,User Rating
0,Painful - how can it get this bad?\n,10-06-2024,Take one of the most exciting and expansive fi...,3.0
1,This is laughably bad writing & direction\n,05-06-2024,Who in the holy hell read these scripts and sa...,3.0
2,What happened to the $180 million it cost to ...,09-06-2024,Then I've seen episode one and two of The Acol...,3.0
3,Jar Jar Binks was a museum masterpiece next t...,13-06-2024,Meesa no lika dis doodoo.I mean seriously what...,1.0
4,Complete garbage\n,12-06-2024,I tried to give this series a chance but after...,1.0


In [4]:
df["review"] = df["Review Title"] + ". " + df["Review Content"]

In [5]:
df.drop(["Review Title", "Review Content"], axis=1, inplace=True)

In [6]:
df.rename(columns={'review': 'Review'}, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493 entries, 0 to 1492
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Review Date  1493 non-null   object 
 1   User Rating  1472 non-null   float64
 2   Review       1493 non-null   object 
dtypes: float64(1), object(2)
memory usage: 35.1+ KB


In [10]:
df.isna().sum()

Review Date     0
User Rating    21
Review          0
dtype: int64

In [11]:
df.dropna()

,Review Date,User Rating,Review
0,10-06-2024,3.0,Painful - how can it get this bad?\n. Take on...
1,05-06-2024,3.0,This is laughably bad writing & direction\n. ...
2,09-06-2024,3.0,What happened to the $180 million it cost to ...
3,13-06-2024,1.0,Jar Jar Binks was a museum masterpiece next t...
4,12-06-2024,1.0,Complete garbage\n. I tried to give this seri...
...,...,...,...
1488,26-06-2024,10.0,Watch first - then rate! Best new SW show sin...
1489,26-06-2024,8.0,This is NOT Star Wars because it is set 100 y...
1490,26-06-2024,10.0,It's hard to get into but wow it paid off in ...
1491,26-06-2024,8.0,Not what it seems- in a great way\n. There is...


In [14]:
tokenizer = DistilBertTokenizer.from_pretrained('bhadresh-savani/distilbert-base-uncased-emotion')
model = DistilBertForSequenceClassification.from_pretrained('bhadresh-savani/distilbert-base-uncased-emotion')

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [16]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)

In [17]:
text=list(df.Review.values)

In [19]:
results = classifier(text, truncation=True, padding=True)

In [53]:
results[0:3]

[[{'label': 'joy', 'score': 0.678463339805603},
  {'label': 'sadness', 'score': 0.29863035678863525},
  {'label': 'anger', 'score': 0.01294006034731865},
  {'label': 'surprise', 'score': 0.004206698387861252},
  {'label': 'love', 'score': 0.0032511220779269934},
  {'label': 'fear', 'score': 0.0025084989611059427}],
 [{'label': 'anger', 'score': 0.9961982369422913},
  {'label': 'fear', 'score': 0.002581368898972869},
  {'label': 'sadness', 'score': 0.00043767160968855023},
  {'label': 'joy', 'score': 0.00033108104253187776},
  {'label': 'love', 'score': 0.0002392815367784351},
  {'label': 'surprise', 'score': 0.0002123247249983251}],
 [{'label': 'sadness', 'score': 0.8747192025184631},
  {'label': 'anger', 'score': 0.0648512989282608},
  {'label': 'joy', 'score': 0.055224765092134476},
  {'label': 'love', 'score': 0.00204348168335855},
  {'label': 'fear', 'score': 0.001963953021913767},
  {'label': 'surprise', 'score': 0.0011973102809861302}]]

In [62]:
def classify_sentiment(emotion_results, threshold=0.6):
    sentiments = []

    positive_emotions = ['joy', 'love', 'surprise']
    negative_emotions = ['sadness', 'anger', 'fear']

    for result in emotion_results:
        max_emotion = max(result, key=lambda x: x['score'])

        if max_emotion['score'] >= threshold:
            if max_emotion['label'] in positive_emotions:
                sentiments.append('positive')
            elif max_emotion['label'] in negative_emotions:
                sentiments.append('negative')
            else:
                sentiments.append('neutral')
        else:
            sentiments.append('neutral')

    return sentiments

# Apply the function to your results
sentiment_classifications = classify_sentiment(results)

# If you want to add these classifications to your DataFrame
df['sentiment'] = sentiment_classifications

In [63]:
df.head(10)

,Review Date,User Rating,Review,sentiment
0,10-06-2024,3.0,Painful - how can it get this bad?\n. Take on...,positive
1,05-06-2024,3.0,This is laughably bad writing & direction\n. ...,negative
2,09-06-2024,3.0,What happened to the $180 million it cost to ...,negative
3,13-06-2024,1.0,Jar Jar Binks was a museum masterpiece next t...,negative
4,12-06-2024,1.0,Complete garbage\n. I tried to give this seri...,negative
5,12-06-2024,1.0,I made an account just to rate this junk\n. T...,positive
6,13-06-2024,1.0,Cultural Vandalism\n. I have never left a one...,negative
7,05-06-2024,NaN,Garbage\n. I'm a huge star wars fan and I can...,negative
8,24-06-2024,8.0,A Solid Entry in the Franchise\n. The Acolyte...,positive
9,13-06-2024,1.0,"The power of 1\n. The power of 2, the power o...",negative
